In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
load_dotenv()


True

## load variables
- data folder
- cleaned_df
- json template
- 

In [2]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
cleaned_df = pd.read_csv(cleaned_csv)
halfmoon_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'cleaned_trees_grasses_halfmoons.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")

with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



### get names from db
the cell has two queries:
- get all somalia users
- get user details for json population

In [3]:
# create a db engine
engine = create_engine(os.getenv('DATABASE_URL1'))

#query 
query = '''select id, username, first_name, last_name, email, country from respi_regreeningusers where country in ('Somalia', 'somalia') order by first_name asc;'''
query2 = '''select id, first_name, last_name, gender, age_category, phone_number, email, country, organization, username from respi_regreeningusers where id in(3228,1831,3726,4247,4548,2495);'''
#execute the query and fetch the results
conn = engine.connect()
df = pd.read_sql_query(query, conn)
df2 = pd.read_sql_query(query2, conn)

#close the connection
conn.close()
engine.dispose()

df2

,id,first_name,last_name,gender,age_category,phone_number,email,country,organization,username
0,1831,Muktar Mohamednoor,Rage,Male,18 – 35,252616051599,mrage@so-actionagainsthunger.org,Somalia,ACF,mrage
1,2495,Hassan Omar,Khadhib,Male,,0615807566,khadhiib10@gmail.com,Somalia,Norwegian Refugee Council,Khadhib
2,3228,Jamaal,Tagal,Male,18 – 35,252616719022,suldaanyare1265@gmail.com,Somalia,ACF,JamalAli
3,3726,Abdiaziz Adan,Hassan,Male,18 – 35,0615704046,abdulazizadan02@gmail.com,Somalia,GREDO,guudow
4,4247,Abdikarim,Mohamed,Male,18 – 35,252617144970,in.qooley@gmail.com,Somalia,GREDO SOMALIA,qoley
5,4548,Abdirashid,Sheikh Mohamed,,,0615197528,abdirashid.kassim@gredosom.org,Somalia,Gredo,NRMO


## enumerator
get user details from db for json population

In [4]:
for idx, i in enumerate(df2.to_dict(orient='records'), start=1):
    i['type'] = 'ENUMERATOR'
    ic(i)

ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'mrage@so-actionagainsthunger.org',
        'first_name': 'Muktar Mohamednoor',
        'gender': 'Male',
        'id': 1831,
        'last_name': 'Rage',
        'organization': 'ACF',
        'phone_number': '252616051599',
        'type': 'ENUMERATOR',
        'username': 'mrage'}
ic| i: {'age_category': '',
        'country': 'Somalia',
        'email': 'khadhiib10@gmail.com',
        'first_name': 'Hassan Omar',
        'gender': 'Male',
        'id': 2495,
        'last_name': 'Khadhib',
        'organization': 'Norwegian Refugee Council',
        'phone_number': '0615807566',
        'type': 'ENUMERATOR',
        'username': 'Khadhib'}
ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'suldaanyare1265@gmail.com',
        'first_name': 'Jamaal',
        'gender': 'Male',
        'id': 3228,
        'last_name': 'Tagal',
        'organization': 'ACF',
        'phone_nu

## farmingEntity
- read column values from cleaned df and append to json keys

In [5]:

#read and set values for farming entity
farming_entity = template[0]['farmingEntity'] 
#ic(farming_entity)

for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    ic(farming_entity)

ic| farming_entity: {'first_name': 'acf',
                     'id': 1,
                     'last_name': 'acf',
                     'middle_name': 'acf',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 2,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 3,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 4,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'acf',
                     'id': 5,
                     'last_name': 'acf',
                   

## plot

In [6]:
import uuid
plot_details = template[0]['plot']

plot_details_json_list = []
count = 0
#assign values from cleaned df    
for i in cleaned_df.to_dict(orient='records'):
    count += 1
    #ic(i)
    plot_details['name'] = uuid.uuid4().hex
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = "https://radrs.icraf.org/uploaded/media/" + i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']
    plot_details['livestock_allowed'] = False if i['land_use_selection-land_use'] == 'agriculture' else True
    plot_details['conservation_area'] = False
    ic(plot_details)
    #ic(count)
    plot_details_json_list.append(plot_details)
#ic(plot_details_json_list)



ic| plot_details: {'calculated_size': 0.469631438,
                   'conservation_area': False,
                   'crops': ['Beans', 'Sorghum', 'Orange'],
                   'farmingEntityId': 119,
                   'fence_type': [],
                   'fenced': False,
                   'has_crops': True,
                   'id': 4,
                   'livestock_allowed': True,
                   'name': '912b80c032e24999aab420cfa1d52f0d',
                   'other_fence_type': '',
                   'other_ownership_type': 'To be specified',
                   'photo_url': 'https://radrs.icraf.org/uploaded/media/1736568888717.jpg',
                   'plot_ownership_type': ['OTHER'],
                   'points': [{'accuracy': 2.812,
                               'altitude': 372.8,
                               'id': 10,
                               'latitude': 3.1870065,
                               'longitude': 43.4571934,
                               'plotId': 4},
     

## plot-points
- parse the geoshape column from cleaned df
- split and extract the values from each row
- append to json keys


In [19]:
def parse_geoshape(geoshape_str):
    points =[]

    #split each lat,lon,alt, acc block
    parts = geoshape_str.split(';')

    for i in parts:
        i = i.strip()
        if not i:
            continue

        lat, lon, alt, acc = map(float, i.split(' '))
        points.append({
            "longitude": lon,
            "latitude": lat,
            "altitude": alt,
            "accuracy": acc
        })
    return points  


def build_points_list(plp_df):
    all_polygons = []
    point_id = 1
    #plp_df plot point dataframe
    
    for idx, row in plp_df.iterrows():
        plot_id = idx + 1

        geoshape_str = row['plot_details-polygon']
        parsed_points = parse_geoshape(geoshape_str)

        points_list = []
        for x in parsed_points:
            points_list.append({
                "id": point_id,
                "plotId": plot_id,
                "longitude": x['longitude'],
                "latitude": x['latitude'],
                "altitude": x['altitude'],
                "accuracy": x['accuracy']
            })
            point_id += 1
        all_polygons.append(points_list)

    return all_polygons

points_json_list = build_points_list(cleaned_df)
        
points_json_list


[[{'id': 1,
   'plotId': 1,
   'longitude': 42.9088313,
   'latitude': 4.2776992,
   'altitude': 267.0,
   'accuracy': 2.6},
  {'id': 2,
   'plotId': 1,
   'longitude': 42.9081516,
   'latitude': 4.2777442,
   'altitude': 269.5,
   'accuracy': 1.371},
  {'id': 3,
   'plotId': 1,
   'longitude': 42.9080836,
   'latitude': 4.2771129,
   'altitude': 267.6,
   'accuracy': 1.4},
  {'id': 4,
   'plotId': 1,
   'longitude': 42.9088917,
   'latitude': 4.2773018,
   'altitude': 270.0,
   'accuracy': 1.4},
  {'id': 5,
   'plotId': 1,
   'longitude': 42.9088313,
   'latitude': 4.2776992,
   'altitude': 267.0,
   'accuracy': 2.6}],
 [{'id': 6,
   'plotId': 2,
   'longitude': 42.3345803,
   'latitude': 2.3993763,
   'altitude': 73.785888671875,
   'accuracy': 3.79},
  {'id': 7,
   'plotId': 2,
   'longitude': 42.3337857,
   'latitude': 2.3989322,
   'altitude': 76.79150390625,
   'accuracy': 5.1},
  {'id': 8,
   'plotId': 2,
   'longitude': 42.3345816,
   'latitude': 2.3973455,
   'altitude': 73.58

## project

In [ ]:
project = template[0]['project']
for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    project['id'] = idx
    project['project_name'] = i['basic_info-organisation_name']
    ic(project)


ic| project: {'id': 1, 'project_name': 'acf'}
ic| project: {'id': 2, 'project_name': 'concern'}
ic| project: {'id': 3, 'project_name': 'concern'}
ic| project: {'id': 4, 'project_name': 'concern'}
ic| project: {'id': 5, 'project_name': 'acf'}
ic| project: {'id': 6, 'project_name': 'concern'}
ic| project: {'id': 7, 'project_name': 'acf'}
ic| project: {'id': 8, 'project_name': 'concern'}
ic| project: {'id': 9, 'project_name': 'concern'}
ic| project: {'id': 10, 'project_name': 'concern'}
ic| project: {'id': 11, 'project_name': 'concern'}
ic| project: {'id': 12, 'project_name': 'concern'}
ic| project: {'id': 13, 'project_name': 'concern'}
ic| project: {'id': 14, 'project_name': 'concern'}
ic| project: {'id': 15, 'project_name': 'concern'}
ic| project: {'id': 16, 'project_name': 'concern'}
ic| project: {'id': 17, 'project_name': 'concern'}
ic| project: {'id': 18, 'project_name': 'concern'}
ic| project: {'id': 19, 'project_name': 'concern'}
ic| project: {'id': 20, 'project_name': 'concern'}
i

## extract repeat csv records for cleaned data
1. get the 'KEY'(works like key and foreign key in the db) values from the main.csv to a list
2. For every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list
3. save the files

In [48]:
#step 1: get the 'KEY' values from cleaned_df to a list
keys = cleaned_df['KEY'].tolist()

#step 2.1: get the other csv filenames
dir = os.path.join(os.getcwd(),'data','brics_csvs')
main_csv = 'Rangeland_Somalia_BRCiSIII.csv'
#halfmoon_csv = 'Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv'
filenames = [file for file in os.listdir(dir) if file.endswith('.csv') and file != main_csv]

#step2.2 : for every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list

for name in filenames:
    #read the repeat csv
    doi = pd.read_csv(os.path.join(dir,name))

    #construct its new name
    new_filename = f'cleaned_{name}'

    #gey the filtered row values for mtching keys
    filtered_doi = doi[doi['PARENT_KEY'].isin(keys)]

    #step3: save the csvs
    filtered_doi.to_csv(os.path.join(dir,'outputs',new_filename), index=False)

    print(f'💯Created: {new_filename} 🎯with {len(filtered_doi)} records')


💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gully.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_contour.csv 🎯with 14 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_swales.csv 🎯with 81 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_keyline.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gabions.csv 🎯with 26 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_terracing.csv 🎯with 6 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_bunds.csv 🎯with 99 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv 🎯with 212 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_basins.csv 🎯with 0 records
💯Created: cleaned_cleaned_trees_grasses_halfmoons.csv 🎯with 212 records


#### halfmoon csv - repeat csv
goal: get plot level information from the repeat csv
- get key for current row
- based on key, in parent key colun, iterate to get all data associated with that key.
- 